In [ ]:
import RPi.GPIO as GPIO
from picamera import PiCamera
import board
import digitalio
from adafruit_bme280 import basic as adafruit_bme280
import pickle
import time
from time import sleep
import math
import cv2
import numpy as np

def contains_blue(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #range for blue 
    hsv_l = np.array([100, 150, 0])
    hsv_h = np.array([140, 255, 255])
    # Find blue pixels in the image
    #
    # cv2.inRange will create a mask (binary array) where the 1 values
    # are blue pixels and 0 values are any other colour out of the blue
    # range defined by hsv_l and hsv_h
    if np.count_nonzero(cv2.inRange(hsv, hsv_l, hsv_h)) > 200:
        return True
    else:
        return False
img_list = []

camera=PiCamera()
start_time = time.time()

GPIO.setmode(GPIO.BCM)
GPIO.setup(17, GPIO.OUT)
GPIO.output(17, GPIO.HIGH)
sleep(2) #might have to increase this to 2
GPIO.output(17, GPIO.LOW)
sleep(2)
try:
    i2c = board.I2C()
    bme280 = adafruit_bme280.Adafruit_BME280_I2C(i2c)
    #bme_cs = digitalio.DigitalInOut(board.D10)
    #bme280 = adafruit_bme280.Adafruit_BME280_SPI(spi, bme_cs) #sea level pressure at FAR is 29.34Hg which is 993.566hPA
    bme280.sea_level_pressure = 993.566
    altitudes_list = []
    img_no = 0
    #add some conditional regarding if the button has been pressed to encompass all this

    init_alt = 0
    for i in range(100):
        calib_alt = bme280.altitude  #bme280 only works with absolute altitude, so we'll establish a baseline on startup at ground
        init_alt+=calib_alt
    init_alt = init_alt/100
    print(init_alt)
                
    while True:
        check_alt = bme280.altitude
        print(check_alt)
        GPIO.output(17, GPIO.HIGH)
        sleep(0.25) #might have to increase this to 2
        GPIO.output(17, GPIO.LOW)
        sleep(0.25)
        #check_alt = 9000 #comment this out
        if check_alt-init_alt > 50: #checks if it has gone high, no point taking pictures the way up   
            check_alt = bme280.altitude
            #check_alt=0 #comment this out
            if check_alt-init_alt < 300: #or whatever feet below which we should start taking photos
                #check_alt = 10000 #comment this out
                while check_alt-init_alt>20:
                    GPIO.output(17, GPIO.HIGH)
                    sleep(0.02) #might have to increase this to 2
                    GPIO.output(17, GPIO.LOW)
                    current_time = time.time()-start_time
                    #camera.capture("/home/pi/Downloads/subscale_test_imgs/img_"+str(img_no)+ "alt: "+str(round(check_alt-init_alt, 4))+"time: "+str(round(current_time,5))+".jpg") #take a picture, for CDR we can probably just save these but we need to get numpy working for analysis
                    camera.resolution = (640, 480)
                    camera.framerate = 24
                    im = np.empty((640,480,3), dtype = np.uint8)
                    camera.capture(im, "rgb")
                    #avoid processing blue
                    if contains_blue(im):
                        continue
                    else:
                        img_list.append(im)
                    img_no+=1
                    altitudes_list.append(["alt"+str(check_alt), "time: "+ str(current_time)])
                    check_alt = bme280.altitude
                    #check_alt = 1000
                break
    #processing
    guesses = []
    acc_count = 0
    sift = cv2.SIFT_create()
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    for i in range(len(img_list)):  #for each pi cam image
        test_keypoints, test_descriptors = sift.detectAndCompute(img_list[i],None)      #calculate descriptor of that image
        matches_counts = []
        if test_descriptors is None:
            print("no descriptor")
            continue
        for j in descriptorslist:                      #compare that discriptor with every descriptor of the 126 classifying images
            #matches = bf.match(j,test_descriptors)  
            matches = flann.knnMatch(j,test_descriptors,k=2)  
            good = []
            for m,n in matches:
                if m.distance < 0.6*n.distance:
                    good.append(m)
            matches_counts.append(len(good))        

        guess = np.argmax(matches_counts)              #return which classifying image most matched the test image
        guesses.append(guess)

    final_mode = stats.mode(guesses)
    final_guess = final_mode[0][0]
    print(final_guess)
    


#so we can look at altitudes corresponding to img#
    with open("altitudes_list", "wb") as fp:
        pickle.dump(altitudes_list,fp)
    with open("altitudes_list", "rb") as fp:
        b=pickle.load(fp)
    print(b)
except:
    print("failed")
    sleep(200)
    for i in range(2000):
        print("picture-taking rn")
        img_no = i
        sleep(2)
        camera.capture("/home/pi/Downloads/subscale_test_imgs/img_%s.jpg" % img_no)